In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

from newscrawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

# 0. 초기화

In [2]:
crawler = NewsCrawler(storage='bigquery')

checking bigquery storage... done (42.86 seconds)
we have total 371169 articles (328816 downloaded, 42353 trashed)


# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [4]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (339.53 seconds): Collecting URLs... indiatimes          

100.00% (0.74 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25781,25717,2920
dailymail,2898,2898,408
reuters,1088,1088,290
investing.com,891,891,175
euronews,1044,1044,171
businessinsider,985,985,137
yahoo,375,375,119
usatoday,401,401,91
rt,417,417,91
aljazeera,524,524,77


,url,pubs,actual_pub
0,https://businessinsider.co.za/capitec-launched...,"businessinsider, news24",businessinsider
1,https://cbsnews.com/news/pete-buttigieg-interv...,"cbs, realclearpolitics",cbs
2,https://foxnews.com/opinion/andy-puzder-left-g...,"fox, realclearpolitics",fox
3,https://hotair.com/archives/ed-morrissey/2019/...,"townhall, hotair",hotair
4,https://medium.com/@teamwarren/ending-the-stra...,"axios, nationalreview",axios
5,https://msnbc.com/msnbc/watch/after-impeachmen...,"msnbc, nbcnews",msnbc
6,https://nationalreview.com/2019/10/remove-the-...,"realclearpolitics, nationalreview",nationalreview
7,https://nationalreview.com/2019/11/documentary...,"realclearpolitics, nationalreview",nationalreview
8,https://nationalreview.com/2019/11/trump-impea...,"realclearpolitics, nationalreview",nationalreview
9,https://nbcnews.com/business/business-news/und...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 크롤링한다

In [5]:
crawl_result = crawler.crawl(); crawl_result

3.90% (132.24 seconds): Crawling... blastingnews        

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (3963.05 seconds): Crawling... dailymail           



collect                                crawl          \
                     collected collected_unique selected downloaded trashed   
all                      25781            25717     2920       1667    1253   
dailymail                 2898             2898      408        196     212   
reuters                   1088             1088      290        129     161   
investing.com              891              891      175         64     111   
euronews                  1044             1044      171         13     158   
businessinsider            985              985      137        100      37   
yahoo                      375              375      119         91      28   
usatoday                   401              401       91         43      48   
rt                         417              417       91         11      80   
aljazeera                  524              524       77         11      66   
metro                      241              241       72         63       9   
fox                        344              344       68         51      17   
news24                    1230             1229       60         60       0   
nbcnews                    244              242       58          1      57   
cnn                        857              857       57         24      33   
axios                      120              119       50         37      13   
breitbart                  415              415       47         46       1   
mirror                     202              202       47         47       0   
marketwatch                185              185       47         31      16   
washingtonpost             120              120       45         42       3   
independent                722              722       45         45       0   
telegraph                  720              720       41         39       2   
zdnet                      344              344       40         27      13   
arynews                    119              119       36         13      23   
thestreet                  132              132       34         17      17   
washingtontimes            460              460       30         28       2   
cbs                        409              409       29         18      11   
wsj                        110              110       28         13      15   
abcnews                     97               97       28         28       0   
indiatimes                 162              162       26         24       2   
...                        ...              ...      ...        ...     ...   
chosun                     297              297       15          0      15   
morningstar                112              112       15         13       2   
dailybeast                  45               45       14          5       9   
msnbc                      149              111       14          3      11   
cbc                        304              304       14         13       1   
hotair                      85               84       12         11       1   
vice                       102              102       11         11       0   
thenextweb                 111              111       11         11       0   
japan-news                  70               70        8          0       8   
nymag                      227              227        8          6       2   
townhall                   622              621        8          7       1   
nytimes                     99               99        7          7       0   
huffpost                    71               71        7          6       1   
cointelegraph               72               72        6          6       0   
politico                   218              218        5          5       0   
atlantic                    96               96        5          5       0   
news.com.au                273              273        4          3       1   
npr                        928              928        4          4      

# 3. 뉴스를 저장한다

In [6]:
crawler.record()

Recording to bigquery storage... done (35.54 seconds)


# 4. 결과보고

In [7]:
crawler.report()

In [66]:
from pathlib import Path
downloaded = [p.stem for p in Path('newsdata/downloaded').glob('**/*.json')]
trashed = [p.stem for p in Path('newsdata/trashed').glob('**/*.json')]

In [65]:
for path in list(Path('newsdata/downloaded').glob('**/*.json'))[:10000]:
    path.unlink()

In [53]:
from pathlib import Path
import json
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header

In [54]:
acc_fname = 'smtp_gmail.json'
acc = json.loads(Path(acc_fname).read_text()); acc

{'host': 'smtp.gmail.com',
 'from': 'gem763@gmail.com',
 'to': ['gem763@gmail.com', 'sejin.kang@hanwha.com'],
 'pw': 'kkangse1'}

In [55]:
msg = MIMEMultipart('alternative')
msg['Subject'] = Header('NEWS CRAWLING REPORT', 'utf-8')
msg['From'] = acc['from']
msg['To'] = ', '.join(acc['to'])
msg.attach(MIMEText(crawler.crawl_summary.to_html(), 'html'))

with smtplib.SMTP_SSL(acc['host']) as smtp:
    smtp.login(acc['from'], acc['pw'])
    smtp.send_message(msg)

In [1]:
import pandas as pd

In [13]:
str(pd.Timestamp.utcnow())[:19]

'2019-11-03 05:39:06'

In [15]:
import json
type(json.load(open('smtp_gmail.json')))

dict